In [10]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
def get_train_times(t1, test_times):
    """
    
    """
    
    trn=t1.copy(deep=True)
    for i, j in test_times.iteritems():
        df0 = trn[(i<=trn.index) & (trn.index<=j)].index # train starts within test
        df1 = trn[(i<=trn) & (trn<=j)].index # train ends within test
        df2 = trn[(trn.index<=i) & (j<=trn)]
        trn = trn.drop(df0.union(df1).union(df2))
    
    return trn

In [3]:
def get_embargo_times(times, pct_embargo):
    
    # get embargo time for each bar
    
    step = int(times.shape[0]*pct_embargo)
    if step==0:
        mbrg=pd.Series(times, index=times)
    else:
        mbrg=pd.Series(times[step:], index=times[:-step])
        mbrg=mbrg.append(pd.Series(times(times[-1], index=times[-step:])))
    
    return mbrg

        

In [13]:
from sklearn.model_selection._split import _BaseKFold

class PurgedKFold(_BaseKFold):
    """
    
    """
    
    def __init__(self, n_splits=3, t1=None, pct_embargo=0.0):
        if not isinstance(t1, pd.Series):
            raise ValueError('Label Through dates must be a pd.Series')
        super(PurgedKFold, self).__init__(n_splits, shuffle=False, random_state=None)
        self.t1=t1
        self.pct_embargo=pct_embargo
        
    def split(self, X, y=None, groups=None):
        if (X.index==self.t1.index).sum() != len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')
        indices = np.arange(X.shape[0])
        mbrg = int(X.shape[0] * self.pct_embargo)
        test_starts = [(i[0], i[-1] + 1) for i in np.array_split(np.arange(X.shape[0]), self.n_splits)]
        
        for i, j in test_starts:
            t0 = self.t1.index[i] # start of test set
            test_indices = indices[i:j]
            max_t1_idx = self.t1.index.searchsorted(self.t1[test_indices].max())
            train_indices = self.t1.index.searchsorted(self.t1[self.t1<=t0].index)
            if max_t1_idx < X.shape[0]: # right train (with embargo)
                train_indices = np.concatenate((train_indices, indices[max_t1_idx + mbrg:]))
            yield train_indices, test_indices
        
                

https://github.com/scikit-learn/scikit-learn/issues/6231